<a href="https://colab.research.google.com/github/atroitskaia/Factory-farming/blob/main/Production_Crops_Livestock__cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [53]:
df = pd.read_csv('/content/drive/MyDrive/DataSchool Project/Climate Change & Farming Data/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_All_Data.csv', encoding='latin1', low_memory=False)

In [54]:
df

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y1961,...,Y2020N,Y2021,Y2021F,Y2021N,Y2022,Y2022F,Y2022N,Y2023,Y2023F,Y2023N
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,ha,0.0,...,NaN,36862.0,A,NaN,36462.00,A,NaN,37000.00,A,NaN
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5412,Yield,kg/ha,NaN,...,NaN,1743.2,A,NaN,1742.00,A,NaN,1810.80,A,NaN
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5510,Production,t,0.0,...,NaN,64256.0,A,NaN,63515.00,A,NaN,67000.00,A,NaN
3,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5312,Area harvested,ha,NaN,...,NaN,25357.0,E,NaN,25403.00,E,NaN,25439.00,E,NaN
4,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5412,Yield,kg/ha,NaN,...,NaN,705.0,E,NaN,704.00,E,NaN,704.00,E,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78167,5817,'902,Net Food Importing Developing Countries,1729,'F1729,"Treenuts, Total",5412,Yield,kg/ha,614.5,...,NaN,683.0,E,NaN,676.60,E,NaN,682.70,E,NaN
78168,5817,'902,Net Food Importing Developing Countries,1729,'F1729,"Treenuts, Total",5510,Production,t,255724.0,...,NaN,2667688.9,E,NaN,2854981.94,E,NaN,2812938.79,E,NaN
78169,5817,'902,Net Food Importing Developing Countries,1735,'F1735,Vegetables Primary,5312,Area harvested,ha,1881425.0,...,NaN,7753692.0,E,NaN,7754677.00,E,NaN,7795058.00,E,NaN
78170,5817,'902,Net Food Importing Developing Countries,1735,'F1735,Vegetables Primary,5412,Yield,kg/ha,6617.1,...,NaN,12136.8,E,NaN,12442.00,E,NaN,12396.10,E,NaN


In [56]:
# Overview
print("Original column names:", df.columns.tolist())

# We need to identify all year columns
year_columns = [col for col in df.columns if col.startswith('Y') and not col.endswith(('F', 'N'))]
print(f"Found {len(year_columns)} year columns from {year_columns[0]} to {year_columns[-1]}")

#Filter for meat-related items and relevant elements
meat_items = [
    'Beef and Buffalo Meat', 'Meat, Poultry', 'Meat, Pig', 'Meat, Sheep',
    'Meat, Goat', 'Meat indigenous', 'Meat, Horse', 'Meat, Ass',
    'Meat, Mule', 'Meat, Camel', 'Meat, Other Camelids', 'Meat, Rabbit',
    'Meat, Other Rodents', 'Meat, Other'
]

# More detailed filter for meat items
meat_filter = df['Item'].apply(lambda x: any(meat_type.lower() in str(x).lower() for meat_type in meat_items))

# Filter rows with meat items and specific elements
meat_rows = df[
    meat_filter &
    df['Element'].isin(['Production', 'Producing Animals/Slaughtered'])
]

# Reshape the data to have years as rows instead of columns
# Select only the necessary columns before melting to reduce memory usage
columns_to_keep = ['Area', 'Item', 'Element', 'Unit'] + year_columns
meat_data_subset = meat_rows[columns_to_keep].copy()

# Melt the data frame to convert year columns to rows
melted_data = pd.melt(
    meat_data_subset,
    id_vars=['Area', 'Item', 'Element', 'Unit'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Value'
)

# Clean up the Year column to remove the 'Y' prefix
melted_data['Year'] = melted_data['Year'].str.replace('Y', '').astype(int)

# Drop rows with missing values (NAN's and FAO)
melted_data = melted_data.dropna(subset=['Value'])

# Filter for US data specifically
us_data = melted_data[melted_data['Area'] == 'United States of America']

# Additional quality checks
print(f"Year range in US data: {us_data['Year'].min()} to {us_data['Year'].max()}")
print(f"Number of unique years in US data: {us_data['Year'].nunique()}")
print(f"Types of meat in US data: {us_data['Item'].unique()}")

# Save processed data
all_meat_data = melted_data.sort_values(['Area', 'Item', 'Element', 'Year'])
all_meat_data.to_csv('/content/drive/MyDrive/DataSchool Project/Clean Data/all_meat_data_clean.csv', index=False)

us_meat_data = us_data.sort_values(['Item', 'Element', 'Year'])
us_meat_data.to_csv('/content/drive/MyDrive/DataSchool Project/Clean Data/us_meat_data_all_years.csv', index=False)

# US animals slaughtered specifically
us_animals_slaughtered = us_data[us_data['Element'] == 'Producing Animals/Slaughtered']
us_animals_slaughtered.to_csv('/content/drive/MyDrive/DataSchool Project/Clean Data/us_animals_slaughtered_all_years.csv', index=False)

# Create a summary dataset with yearly totals for quick visualization
yearly_totals = us_animals_slaughtered.groupby(['Year', 'Item'])['Value'].sum().reset_index()
yearly_totals.to_csv('/content/drive/MyDrive/DataSchool Project/Clean Data/us_yearly_slaughter_by_animal.csv', index=False)

print("Data processing complete!")

Original column names: ['Area Code', 'Area Code (M49)', 'Area', 'Item Code', 'Item Code (CPC)', 'Item', 'Element Code', 'Element', 'Unit', 'Y1961', 'Y1961F', 'Y1961N', 'Y1962', 'Y1962F', 'Y1962N', 'Y1963', 'Y1963F', 'Y1963N', 'Y1964', 'Y1964F', 'Y1964N', 'Y1965', 'Y1965F', 'Y1965N', 'Y1966', 'Y1966F', 'Y1966N', 'Y1967', 'Y1967F', 'Y1967N', 'Y1968', 'Y1968F', 'Y1968N', 'Y1969', 'Y1969F', 'Y1969N', 'Y1970', 'Y1970F', 'Y1970N', 'Y1971', 'Y1971F', 'Y1971N', 'Y1972', 'Y1972F', 'Y1972N', 'Y1973', 'Y1973F', 'Y1973N', 'Y1974', 'Y1974F', 'Y1974N', 'Y1975', 'Y1975F', 'Y1975N', 'Y1976', 'Y1976F', 'Y1976N', 'Y1977', 'Y1977F', 'Y1977N', 'Y1978', 'Y1978F', 'Y1978N', 'Y1979', 'Y1979F', 'Y1979N', 'Y1980', 'Y1980F', 'Y1980N', 'Y1981', 'Y1981F', 'Y1981N', 'Y1982', 'Y1982F', 'Y1982N', 'Y1983', 'Y1983F', 'Y1983N', 'Y1984', 'Y1984F', 'Y1984N', 'Y1985', 'Y1985F', 'Y1985N', 'Y1986', 'Y1986F', 'Y1986N', 'Y1987', 'Y1987F', 'Y1987N', 'Y1988', 'Y1988F', 'Y1988N', 'Y1989', 'Y1989F', 'Y1989N', 'Y1990', 'Y1990F', '

In [57]:
df_1 = pd.read_csv('/content/drive/MyDrive/DataSchool Project/Clean Data/all_meat_data_clean.csv')
df_1

,Area,Item,Element,Unit,Year,Value
0,Afghanistan,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1961,360000.00
1,Afghanistan,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1962,384000.00
2,Afghanistan,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1963,396000.00
3,Afghanistan,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1964,402000.00
4,Afghanistan,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1965,408000.00
...,...,...,...,...,...,...
54848,Zimbabwe,"Meat, Poultry",Production,t,2019,114534.09
54849,Zimbabwe,"Meat, Poultry",Production,t,2020,111725.00
54850,Zimbabwe,"Meat, Poultry",Production,t,2021,113178.29
54851,Zimbabwe,"Meat, Poultry",Production,t,2022,118405.57


In [58]:
df_2 = pd.read_csv('/content/drive/MyDrive/DataSchool Project/Clean Data/us_meat_data_all_years.csv')
df_2

,Area,Item,Element,Unit,Year,Value
0,United States of America,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1961,34551008.00
1,United States of America,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1962,34768000.00
2,United States of America,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1963,35274000.00
3,United States of America,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1964,39310000.00
4,United States of America,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1965,40959008.00
...,...,...,...,...,...,...
247,United States of America,"Meat, Poultry",Production,t,2019,22899942.45
248,United States of America,"Meat, Poultry",Production,t,2020,21471397.00
249,United States of America,"Meat, Poultry",Production,t,2021,21807666.00
250,United States of America,"Meat, Poultry",Production,t,2022,22272253.00


In [59]:
df_3 = pd.read_csv('/content/drive/MyDrive/DataSchool Project/Clean Data/us_animals_slaughtered_all_years.csv')
df_3

,Area,Item,Element,Unit,Year,Value
0,United States of America,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1961,34551008.0
1,United States of America,"Meat, Poultry",Producing Animals/Slaughtered,1000 An,1961,2366872.0
2,United States of America,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1962,34768000.0
3,United States of America,"Meat, Poultry",Producing Animals/Slaughtered,1000 An,1962,2379525.0
4,United States of America,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,1963,35274000.0
...,...,...,...,...,...,...
121,United States of America,"Meat, Poultry",Producing Animals/Slaughtered,1000 An,2021,9565548.0
122,United States of America,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,2022,34811900.0
123,United States of America,"Meat, Poultry",Producing Animals/Slaughtered,1000 An,2022,9781104.0
124,United States of America,"Beef and Buffalo Meat, primary",Producing Animals/Slaughtered,An,2023,33254800.0


In [60]:
df_4 = pd.read_csv('/content/drive/MyDrive/DataSchool Project/Clean Data/us_yearly_slaughter_by_animal.csv')

In [61]:
df_4

,Year,Item,Value
0,1961,"Beef and Buffalo Meat, primary",34551008.0
1,1961,"Meat, Poultry",2366872.0
2,1962,"Beef and Buffalo Meat, primary",34768000.0
3,1962,"Meat, Poultry",2379525.0
4,1963,"Beef and Buffalo Meat, primary",35274000.0
...,...,...,...
121,2021,"Meat, Poultry",9565548.0
122,2022,"Beef and Buffalo Meat, primary",34811900.0
123,2022,"Meat, Poultry",9781104.0
124,2023,"Beef and Buffalo Meat, primary",33254800.0
